# Tests

In [ ]:
%load_ext autoreload
%autoreload 2

from utils_postprocess import printDict

import globvars as gv
import mne
import matplotlib.pyplot as plt
from os.path import join as pjoin


defdgen = lambda sz :np.random.uniform(low=-1,high=1, size=sz)

import utils_tSNE as utsne

sfreq = 256
sfreq_hires = 1024

from IPython import get_ipython; ipython = get_ipython()
ipython.magic('run -i ../test_data/gen_test_data4.py')

print(test_data_descr)

#rawnames = ['S01_off_move','S01_off_hold','S01_on_move']

# prep_test_data

# _gt means ground truth here
dat_pri_gt           = dat_pri
dat_LFP_hires_pri_gt = dat_LFP_hires_pri

plt.plot(times,dat_pri[dati].T)
plt.figure()
plt.plot(times_hires,dat_LFP_hires_pri[dati].T, label=sfo_LFP)
plt.legend()

dat_EMG = defdgen( (2, nbins ) )

# reversed if needed
from utils import makeSimpleRaw
from featlist import selFeatsRegexInds
raws_permod_both_sides2 = {}
for rawi,rawn in enumerate(rawnames):
    raws_permod_both_sides2[rawn] = {}
    chis_LFP = utsne.selFeatsRegexInds(sfo_pri[rawi], 'LFP.*' )
    dat_LFP = dat_pri_gt[rawi][chis_LFP]
    raws_permod_both_sides2[rawn]['LFP'] = makeSimpleRaw(dat_LFP, np.array(sfo_pri[rawi])[chis_LFP], rescale=0 )

    chis_msrc = utsne.selFeatsRegexInds(sfo_pri[rawi], 'msrc.*' )
    dat_msrc = dat_pri_gt[rawi][chis_msrc]
    raws_permod_both_sides2[rawn]['src'] = makeSimpleRaw(dat_msrc, 
        np.array(sfo_pri[rawi])[chis_msrc] , rescale=0)

    raws_permod_both_sides2[rawn]['LFP_hires'] = makeSimpleRaw(dat_LFP_hires_pri_gt[rawi],
        sfo_lfp_hires_pri[rawi], sfreq=sfreq_hires, rescale=0 )
    
    raws_permod_both_sides2[rawn]['EMG'] = makeSimpleRaw(dat_EMG, 
        gv.EMG_per_hand[main_side], sfreq=sfreq, rescale=0)

rawnames_str = ','.join(rawnames)

####################################
import globvars as gv
gv.DEBUG_MODE=1
from IPython import get_ipython; ipython = get_ipython()
#Once that's done, run a magic command like this:
#%debug
raws_permod_both_sides = raws_permod_both_sides2

mstr = 'run -i ../run/run_prep_dat.py -r ' +   rawnames_str + \
        ' --param_file prep_dat_defparams.ini --calc_stats_multi_band 1' +\
        ' --save_dat 1 --save_stats 1 --use_preloaded_raws 1 --allow_CUDA 0'
ipython.magic(mstr)

# %run -i ../run/run_prep_dat.py -r S99_off_move,S99_off_hold,S99_on_move \
#         --param_file prep_dat_defparams.ini --calc_stats_multi_band 1 --save_dat 1 \
#         --use_preloaded_raws 1 --save_stats 1

gv.DEBUG_MODE = True
from IPython import get_ipython; ipython = get_ipython()
#debug

# import warnings
# warnings.filterwarnings("error")
# # with warnings.catch_warnings():
# #     warnings.simplefilter('error')

# try:

# except ComplexWarning as e:
#     pass

rs = ('%run -i ../run/run_genfeats.py '
' --sources_type parcel_aal --bands crude'
' --src_grouping_fn 10'
' --src_grouping 0 '
' --raw {} '
' --feat_types con,H_act,H_mob,H_compl,rbcorr,bpcorr'
' --Kalman_smooth 0'
' --load_TFR 0'
' --load_CSD 0'
' --save_TFR 0'
' --save_CSD 0'
' --save_bpcorr 0'
' --save_rbcorr 0'
' --load_rbcorr 0'
' --use_existing_TFR 0'
' --use_preloaded_data 0'
' --allow_CUDA 0'
' --load_only 0'
' --show_plots 0'
' --plot_types ,'
' --prescale_data 1'
' --exit_after {}'
' --normalize_TFR separate'
' --scale_data_combine_type no'
' --baseline_int_type entire'
' --n_jobs 1'
' --save_feat 0'
' --feat_stats_artif_handling reject'
' --scale_data_combine_type no '
' --rbcorr_use_local_means 0'
' --output_subdir test'
' --stats_fn_prefix stats_{}_{}_ '
' --param_file genfeats_defparams.ini ')

exit_after = 'end'
#exit_after = 'prescale_data'
#exit_after = 'load'
mstr2 = rs.format(rawnames_str,exit_after,rawnames_str[:3],len(rawnames))

#%debug

ipython.magic(mstr2)

#np.savez(pjoin(gv.data_dir,'test.npz') ,globals() )

# TFR

In [ ]:
        csd_cur, csdord = ugf.tfr2csd(tfrres_pri[0], sfreq, returnOrder=1,
                                        ind_pairs=None,
                                        parcel_couplings=parcel_couplings,
                                        LFP2LFP_couplings=LFP2LFP_couplings,
                                        LFP2parcel_couplings=LFP2parcel_couplings,
                                        oldchns=chnames_tfr,
                                        newchns=newchns,
                                        res_group_id=newchn_grouping_ind,
                                        log=log_during_csd)


In [ ]:
len(freqs),len(freqs_inc_HFO), len(freqs_inc_HFO)-len(freqs)

In [ ]:
tfrres_LFP_HFO_pri[0].shape

In [ ]:
from plots import plotTFRlike

p = {'dat_pri':dat_pri, 'dat_hires_pri':dat_lfp_hires_pri,  
     'tfrres_pri':tfrres_pri, 'tfrres_HFO_pri':tfrres_LFP_HFO_pri, 'csd_pri':csd_pri,
    'bpow_abscsd_pri':bpow_abscsd_pri,
    'wbd':wbd_H_pri, 'feat_dict':feat_dict,
    'times_pri':times_pri, 'times_hires_pri':times_hires_pri,
    'subfeature_order_pri':subfeature_order_pri, 
     'subfeature_order_newsrcgrp_pri':subfeature_order_newsrcgrp_pri,
    'csdord_pri':csdord_pri, 'csdord_strs_pri':csdord_strs_pri,
    'freqs':freqs_inc_HFO, 'sfreq':sfreq, 
    'normalize':True}

#%debug
fbs = ['tremor','beta', 'gamma']
for chn_cur in subfeature_order_pri[0]:
    chns = [chn_cur]
    funloc = plotTFRlike(**p, bands=fbs, chns=chns)
    plt.savefig(pjoin(gv.dir_fig,f'tfrplot_{chns}.pdf' ) )
    plt.close()

#chns = ['msrcR_0_3_c5', 'LFPR092']
chns = subfeature_order_pri[0]
chns_cur = chns[:2]
funloc = plotTFRlike(**p, bands=fbs, chns=chns_cur)
plt.savefig(pjoin(gv.dir_fig,f'tfrplot_{chns_cur}.pdf' ) )
plt.close()


chns = subfeature_order_pri[0]
chns_cur = chns[:2]
funloc = plotTFRlike(**p, bands=fbs, chns=chns_cur)
plt.savefig(pjoin(gv.dir_fig,f'tfrplot_{chns_cur}.pdf' ) )
plt.close()



In [ ]:
binom(6,1)

In [ ]:
freqs_inc_HFO[47:]

In [ ]:
np

In [ ]:
tfrres_pri[0].shape, tfrres_LFP_HFO_pri[0].shape

In [ ]:
recalc_stats_multi_band

In [ ]:
noise_size

In [ ]:
# Check the we have scaled correctly
m0 = max( np.max( dat_pri_unscaled[0] - dat_pri_unscaled[1] ), np.max( dat_pri_unscaled[0] - dat_pri_unscaled[2] ) )
m1 = max( np.max( dat_pri[0] - dat_pri[1] ), np.max( dat_pri[0] - dat_pri[2] ) )
print(m0,m1)
assert m1 < noise_size

# Check the we have scaled correctly
m0 = max( np.max( dat_lfp_hires_pri_unscaled[0] - dat_lfp_hires_pri_unscaled[1] ), 
         np.max( dat_lfp_hires_pri_unscaled[0] - dat_lfp_hires_pri_unscaled[2] ) )
m1 = max( np.max( dat_lfp_hires_pri[0] - dat_lfp_hires_pri[1] ), 
         np.max( dat_lfp_hires_pri[0] - dat_lfp_hires_pri[2] ) )
print(m0,m1)
assert m1 < noise_size

In [ ]:
np.sum( np.isnan(C_nocenter) )

In [ ]:
plt.pcolor( np.cov(X_pri[rawi],rowvar=False) )
plt.figure()
plt.hist( np.cov(X_pri[rawi] ).flatten() )

In [ ]:
corr_info_pri  = len(rawnames ) * [0]
C_pri = len(rawnames ) * [0]
for rawi in range(len(rawnames)):
    C = np.corrcoef(X_pri[rawi],rowvar=False)
    
    absC = np.abs(C)

    C_nocenter = absC - np.diag(np.diag(absC))
    C_pri[rawi] = C_nocenter

    C_flat = C_nocenter.flatten()
    sinds = np.argsort(C_flat)
    hist, bin_edges = np.histogram(C_nocenter.flatten(), bins=20, density=False)
    
    info = {'C':C, 'C_nocenter':C_nocenter, 'hist':hist, 'bin_edges':bin_edges}
    corr_info_pri[rawi] = info



Cs = np.stack(C_pri, axis=-1)
Cmean = np.mean(Cs,axis=-1)

import utils_postprocess as postp
#q = 1-1e-2
q = 0.99
#scft = postp.getStrongCorrelPairs(C_pri[0],q)
scft = postp.getStrongCorrelPairs(Cmean,q)
for i,j in scft:
    print(feature_names_all[i],feature_names_all[j])

In [ ]:
plt.pcolor(Cmean)

plt.figure()
plt.plot(bin_edges[1:],hist)

In [ ]:
from featlist import parseFeatNames

In [ ]:
subfeature_order

In [ ]:
gv.fband_names_crude

In [ ]:
fband_names_inc_HFO

In [ ]:
import featlist
#     if cond_ft and cond_bands:
#         print(feature_names_all[fi],fbs_cur,chns_cur, fts,relevant_freqs,cond_ft,cond_bands,cond_chns)
featlist.selectFeatNames(fts,relevant_freqs,desired_chns, feature_names_all, fband_names=fband_names_inc_HFO)
featlist.replaceMEGsrcChnamesParams(desired_chns, 0,9, '.*', 0)

In [ ]:
# Plot together: raw data, features and intermediate stuff (except spectral)

# Plot meaningful channels and features together

In [ ]:
test_plots_descr

In [ ]:
from featlist import parseFeatNames
from featlist import selectFeatNames
from utils import freqs2relevantBands
featnames_parse_res = parseFeatNames( feature_names_all)
featnames_parse_res.keys()

from plots import plotFeatsAndRelDat
from plots import plotDataAnnStat
from plots import shadeAnn
from plots import plotMeansPerIt

#%debug
for tpd in test_plots_descr:
#for tpd in test_plots_descr[:1]:
#for tpd in test_plots_descr[0:1]:
#for tpd in test_plots_descr[1:2]:
#for tpd in test_plots_descr[2:3]:
    
    from matplotlib.backends.backend_pdf import PdfPages
    #fign = rawnstr + '__rawdata_vs_feat'
    fign = tpd['figname'] + '__rawdata_vs_feat'
    figfn_full = pjoin(gv.dir_fig, fign+'.pdf')
    #plt.savefig(figfn_full)
    pdf= PdfPages(figfn_full  )
    
    
    ds = tpd['chn_descrs']
    print('   ',tpd['informal_descr'])
    desired_chns = [special_chns[chnd] for chnd in ds]
    fts = tpd['feat_types_actual_coupling']
    relevant_freqs = tpd['relevant_freqs']
    print(ds,desired_chns)
    print(fts,relevant_freqs)
    
    featnames_sel = selectFeatNames(fts,relevant_freqs,desired_chns, feature_names_all,
                                    fband_names=fband_names_inc_HFO)
    print(featnames_sel)
    
    # select features related to these channels
    plotFeatsAndRelDat(rawnames, featnames_sel, dat_pri,subfeature_order_pri,                         
                       X_pri,[feature_names_all]*3,times_pri,Xtimes_pri,
                      subfeature_order_newsrcgrp_pri, wbd_H_pri,
                      dat_hires_pri=dat_lfp_hires_pri,
                       chnames_all_hires_pri = subfeature_order_lfp_hires_pri,
                       times_hires_pri=times_hires_pri, 
                       anndict_per_intcat_per_rawn=anndict_per_intcat_per_rawn, sfreq=sfreq )
    plt.tight_layout()
    pdf.savefig()
    plt.close()


#ct = 'medcond'

    tpl1 = (raw_perband_flt_pri,stats_multiband_flt,'flt')
    tpl2 = (raw_perband_bp_pri, stats_multiband_bp,'bp')
    tpls = (tpl1,tpl2)
    for dat_dict,stat_dict,dat_type in tpls:    
        bands_cur = freqs2relevantBands(relevant_freqs, fband_names_inc_HFO)
        for band in bands_cur:
        #for band in ['HFO']:
        #for band in ['beta']:
        #band = 'gamma'
        #band = 'beta'
        #band = 'tremor'        
            means_per_iset = stat_dict['means'].get(band, None)
            indsets = stat_dict['indsets']
            suptitle = f'combin={scale_data_combine_type}  band={band} dat_type={dat_type}'

            if means_per_iset is None:
                continue

            r = parseFeatNames(featnames_sel)
            chnames_involved = [chn for chn in (r['ch1'] + r['ch2']) if chn is not None]
            chnames_involved = list(set(chnames_involved))
            chis = [subfeature_order_newsrcgrp_pri[0].index( chn ) for chn in chnames_involved]

            plotDataAnnStat(rawnames, dat_pri, times_pri, subfeature_order_pri,  
                               dat_lfp_hires_pri,times_hires_pri,subfeature_order_lfp_hires_pri,
                               anndict_per_intcat_per_rawn,
                               indsets,means_per_iset,suptitle=suptitle,
                               dat_dict=dat_dict,band=band,legend_loc='upper left',
                           chis_to_show = chis)
            plt.tight_layout()
            pdf.savefig()
            plt.close()

    pdf.close()

In [ ]:
subfeature_order_newsrcgrp_pri

In [ ]:
subfeature_order_pri

In [ ]:
noise_size

In [ ]:
freqs

In [ ]:
nlowfreq

In [ ]:
#subfeature_order_newsrcgrp_pri

In [ ]:
subfeature_order

In [ ]:
freqs >= fbands[band][0]

In [ ]:
band='beta'
freqis = np.where( (freqs >= fbands[band][0]) & (freqs <= fbands[band][1]) )[0]
freqs[freqis]

In [ ]:
subfeature_order_newsrcgrp_pri

In [ ]:
import re
regex = 'beta_(msrcR_0_3_c5|LFPR092),(msrcR_0_3_c5|LFPR092)'
r = re.match( 'beta_(msrcR_0_3_c5|LFPR092),(msrcR_0_3_c5|LFPR092)' , 'beta_msrcR_9_3_c0,msrcR_9_3_c0')
print(r)
selFeatsRegexInds( feat_dict['con']['names'], regex )

In [ ]:
feat_dict['con']['names']

In [ ]:
bb = 'beta'
fbi = fband_names.index(bb)
for csdi in range(bpow_abscsd_pri[0].shape[0]):
    plt.figure(figsize=(15,2)); ax = plt.gca()
    ax.plot(Xtimes, np.abs( bpow_abscsd_pri[rawi][csdi,fbi] ) )
    ax.set_title(f'{csdi}  {csdord_pri[0][:,csdi] }   {csdord_strs_pri[0][csdi]}')

In [ ]:
AA = bpow_abscds_reshaped
for i,s in enumerate(csdord_strs_pri[0]):
    plt.figure(figsize=(15,2)); ax = plt.gca()
    ax.plot(Xtimes,AA[i])
    #ii,jj = np.unravel_index(i,bpow_abscsd.shape[:2])
    #print(i,ii,jj)
    #i1,i2 = csdord[:,i]
    #print(i, csdord[:,ii], ii,jj)
    ax.set_title(f'{s} ') #--  {chnames_tfr[i1]},{chnames_tfr[i2]}')


In [ ]:
ab

In [ ]:
bpow_abscsd.shape

In [ ]:
tfrres.shape[1]

In [ ]:
tfrres_LFP_HFO.shape[1]

In [ ]:
t

In [ ]:
isinstance(6,(float,int))

In [ ]:
stats_multiband_bp

# See why correlated features arise

In [ ]:
import utils_postprocess as postp
scft = postp.getStrongCorrelPairs(Cmean,0.9999)

scft = postp.getStrongCorrelPairs(C_pri[0],0.92)
for i,j in scft:
    print(feature_names_all[i],feature_names_all[j])

from utils_postprocess import getSynonymList
lll = getSynonymList(scft, ret_dict = 0)
print(lll)

In [ ]:
ll = lll[1]
syns_to_disp = [feature_names_all[l] for l in ll]
print( syns_to_disp )

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
fign = rawnstr + '__rawdata_vs_feat_corrfeat'
figfn_full = pjoin(gv.dir_fig, fign+'.pdf')
#plt.savefig(figfn_full)
pdf= PdfPages(figfn_full  )


from featlist import parseFeatNames
featnames_parse_res = parseFeatNames( feature_names_all_pri[0])
featnames_parse_res.keys()

from plots import plotFeatsAndRelDat
from plots import plotDataAnnStat
from plots import shadeAnn
from plots import plotMeansPerIt
from featlist import selectFeatNames
from utils import freqs2relevantBands

#for i,j in scft[:1]:
#for i,j in [syns_to_disp]:
featnames_sel = syns_to_disp

bands_cur = set([])
for fn in syns_to_disp:
    i = feature_names_all_pri[0].index(fn) 
    bands_cur |= set([featnames_parse_res['fb1'][i], featnames_parse_res['fb2'][i]])

bands_cur = list(bands_cur)
#featnames_sel =[ feature_names_all[i],feature_names_all[j]]

# select features related to these channels
plotFeatsAndRelDat(rawnames, featnames_sel, dat_pri,subfeature_order_pri,                         
                   X_pri,[feature_names_all]*3,times_pri,Xtimes_pri,
                  subfeature_order_newsrcgrp_pri, wbd_H_pri,
                  dat_hires_pri=dat_lfp_hires_pri,
                   chnames_all_hires_pri = subfeature_order_lfp_hires_pri,
                   times_hires_pri=times_hires_pri, 
                   anndict_per_intcat_per_rawn=anndict_per_intcat_per_rawn, sfreq=sfreq )
plt.tight_layout()
pdf.savefig()
plt.close()


#ct = 'medcond'

tpl1 = (raw_perband_flt_pri,stats_multiband_flt,'flt')
tpl2 = (raw_perband_bp_pri, stats_multiband_bp,'bp')
tpls = (tpl1,tpl2)
for dat_dict,stat_dict,dat_type in tpls:        
    for band in bands_cur:
    #for band in ['HFO']:
    #for band in ['beta']:
    #band = 'gamma'
    #band = 'beta'
    #band = 'tremor'        
        means_per_iset = stat_dict['means'].get(band, None)
        indsets = stat_dict['indsets']
        suptitle = f'combin={scale_data_combine_type}  band={band} dat_type={dat_type}'
        
        if means_per_iset is None:
            continue
            
        r = parseFeatNames(featnames_sel)
        chnames_involved = [chn for chn in (r['ch1'] + r['ch2']) if chn is not None]
        chnames_involved = list(set(chnames_involved))
        chis = [subfeature_order_newsrcgrp_pri[0].index( chn ) for chn in chnames_involved]

        plotDataAnnStat(rawnames, dat_pri, times_pri, subfeature_order_pri,  
                           dat_lfp_hires_pri,times_hires_pri,subfeature_order_lfp_hires_pri,
                           anndict_per_intcat_per_rawn,
                           indsets,means_per_iset,suptitle=suptitle,
                           dat_dict=dat_dict,band=band,legend_loc='upper left',
                       chis_to_show = chis)
        plt.tight_layout()
        pdf.savefig()
        plt.close()
        
pdf.close()

In [ ]:
if bands_acc == 'crude':
    fbl = gv.fband_names_crude
else:
    fbl = gv.fband_names_fine
desired_fbands = []
for fbname in fbl:
    fb = gv.fbands[fbname]
    t = (rf >= fb[0]) and (rf <= fb[1])
    if t:
        desired_fbands += [fbname]

In [ ]:
feat

In [ ]:
ipdb> featn
'con_tremor_LFPR092,msrcR_9_3_c0'
ipdb> featnames_all_pri[rawi]
'con_tremor_msrcR_9_2_c0,msrcR_9_2_c0'

### check scaling with plots

In [ ]:
ax = plt.gca()
d = dat_pri[0]
ts = times_pri[0]
ax.plot(ts,d[0], label='non scaled')
d = dat_pri[1]
ts = times_pri[1]
ax.plot(ts,d[0], label='procued by my code')
plt.legend()

plt.figure()

ax = plt.gca()
d = dat_pri[0]
ts = times_pri[0]
ax.plot(ts,d[0], label='non scaled')
d = dat_pri[1]
ts = times_pri[1]
ax.plot(ts,d[0]*10, label='scaled here')
plt.legend()

In [ ]:
newindsets

In [ ]:
means

In [ ]:
baseline_int

In [ ]:
scale_data_combine_type

In [ ]:
dat_pri_scaled = len(dat_pri) * [0]
for dati in range(len(dat_pri) ):
    dat_pri_scaled[dati] = dat_T_scaled[dati].T
max( np.max( dat_pri_scaled[0] - dat_pri_scaled[1] ), np.max( dat_pri_scaled[0] - dat_pri_scaled[2] ) )

In [ ]:
nr= 1 ;
nc =len(rawnames)
ww = 6; hh=4
fig,axs = plt.subplots(nr,nc, figsize=(nc*ww,nr*hh) )
for dati in range(len(rawnames)):
    ax = axs[dati]
    ts = times_pri[dati]
    d = dat_pri_unscaled[dati]    
    #ax.plot(ts,d[0],label=f'orig data {dati}' )
    m = means[dati]['entire'][0]
    s = stds[dati]['entire'][0]
    #ax.plot(ts,(d[0] - m) / s,label=f'scaled here {dati}')
    ax.plot(ts,d[0] / data_mult_per_rawi[dati],label=f'scaled gt {dati}')
    
    d = dat_pri[dati]
    ax.plot(ts,d[0],label=f'scaled {dati}')
    
#     ax.axhline(m, ls='-.', c='purple',label='mean')
#     ax.axhline(m-s, ls=':', c='purple',label='std')
#     ax.axhline(m+s, ls=':', c='purple')
    ax.set_title(f'm={m:.4f}, std={s:.4f}')
    ax.legend()

In [ ]:
nr= 1 ;
nc =len(rawnames)
ww = 6; hh=4
chi = 1
fig,axs = plt.subplots(nr,nc, figsize=(nc*ww,nr*hh) )
for dati in range(len(rawnames)):
    ax = axs[dati]
    ts = times_hires_pri[dati]
    d = dat_lfp_hires_pri_unscaled[dati]    
    #ax.plot(ts,d[0],label=f'orig data {dati}' )
    m = means[dati]['entire'][chi]
    s = stds[dati]['entire'][chi]
    #ax.plot(ts,(d[0] - m) / s,label=f'scaled here {dati}')
    ax.plot(ts,d[chi] / data_mult_per_rawi[dati],label=f'scaled gt {dati}')
    
    d = dat_lfp_hires_pri[dati]
    ax.plot(ts,d[chi],label=f'scaled {dati}')
    
#     ax.axhline(m, ls='-.', c='purple',label='mean')
#     ax.axhline(m-s, ls=':', c='purple',label='std')
#     ax.axhline(m+s, ls=':', c='purple')
    ax.set_title(f'm={m:.4f}, std={s:.4f}')
    ax.legend()

In [ ]:
list(f.keys())

In [ ]:
    recalc_stats_multi_band = 1
    
    if recalc_stats_multi_band:
        #means_perband_flt_pri_,_ = \
        indsets, means_perband_flt_pri_, stds_perband_flt_pri_, stats_per_indset_per_band_flt =  \
        ugf.gatherMultiBandStats(rawnames,raw_perband_flt_pri, times_pri,
                                chnames_perband_flt_pri, side_switched_pri, sfreq,
                                baseline_int, scale_data_combine_type,
                                feat_stats_artif_handling,
                                require_intervals_present = [baseline_int],
                                 bindict_per_rawn=bindict_per_rawn)
        #means_perband_bp_pri_,_ = \
        indsets, means_perband_bp_pri_, stds_perband_bp_pri_, stats_per_indset_per_band_bp =  \
        ugf.gatherMultiBandStats(rawnames,raw_perband_bp_pri, times_pri,
                                chnames_perband_bp_pri, side_switched_pri, sfreq,
                                baseline_int, scale_data_combine_type,
                                feat_stats_artif_handling,
                                require_intervals_present = [baseline_int],
                                 bindict_per_rawn=bindict_per_rawn)
    else:
        prefix = stats_fn_prefix
        fname_stats_multi_band = utils.genStatsMultiBandFn(None, new_main_side, data_modalities,
                                        use_main_LFP_chan, src_file_grouping_ind,
                                        src_grouping, bands_only, prefix )
        fname_stats_full = pjoin( gv.data_dir, input_subdir, fname_stats_multi_band)
        f = np.load(fname_stats_full, allow_pickle=True)
        rawnames_stats =  f['rawnames']
        assert set(rawnames).issubset(rawnames_stats)
        #means_perband_flt_pri_ =  f['stats_multiband_flt_per_ct'][()][scale_data_combine_type]['means']
        #means_perband_bp_pri_  =  f['stats_multiband_bp_per_ct'][()][scale_data_combine_type]['means']

        stats_multiband_flt =  f['stats_multiband_flt_per_ct'][()][scale_data_combine_type]
        stats_multiband_bp  =  f['stats_multiband_bp_per_ct'][()][scale_data_combine_type]

        #indsets = stats_multiband_bp['indsets']

        #stats_per_indset_per_band_flt

In [ ]:
stats_multiband_flt = stats_per_indset_per_band_flt
stats_multiband_bp = stats_per_indset_per_band_bp 

In [ ]:
stats_multiband_flt

In [ ]:
    ################# extract stats for the given baseline int

    rawi_mask,allinds = upre.getIndsetMask(indsets, allow_repeating=False, allow_holes = False)

    means_perband_flt_pri = len(rawnames) * [dict()]
    means_perband_bp_pri = len(rawnames) * [dict()]

    statdicts =[stats_multiband_flt, stats_multiband_bp ]
    mdicts =[means_perband_flt_pri, means_perband_bp_pri ]
    for mdi,md in enumerate(mdicts):
        for rawind in range(len(rawnames)):
            indseti = rawi_mask[rawind]
            if recalc_stats_multi_band:
                for bandname in statdicts[mdi]:
                    a = statdicts[mdi][bandname][indseti][baseline_int]
                    #print(a)
                    mdicts[mdi][rawind][bandname] = a
            else:
                for bandname in statdicts[mdi]['means']:
                    a = statdicts[mdi]['means'][bandname][indseti][baseline_int]
                    #print(a)
                    mdicts[mdi][rawind][bandname] = a


In [ ]:
rawi_mask

In [ ]:
indseti

In [ ]:
statdicts[mdi]['means'][bandname]

In [ ]:
baseline_int

In [ ]:
statdicts[mdi]['means'][bandname][0][baseline_int]

In [ ]:
statdicts[mdi][bandname]

In [ ]:
stats_fn_prefix

In [ ]:
utils.genStatsMultiBandFn(None, new_main_side, data_modalities,
                                        use_main_LFP_chan, src_file_grouping_ind,
                                        src_grouping, bands_only, prefix )

In [ ]:
fname_stats_full

In [ ]:
#Saving multiband stats  /home/demitau/data/stats_S95_3_crude_dat_LFP,msrc_newmsL_mainLFP0_grp10-0.npz